In [18]:
# Partilha de chave Shamir

# Inicialização
def main(bits):
    """
Função principal que invoca as outras.
Input: número de bits no primo p.
Output: retorna os parametros n, k, g, p, a0, onde
n-total de participantes ao esquema, k-elementos necessários para reconstruir
o segredo, g-gerador e a0 o segredo.
    """
    p= random_prime(2**bits)
    Zp= IntegerModRing(p)
    g=Zp.multiplicative_generator()
    a0=input("Digite o segredo a ser compartilhado:")
    n=input("Quantos participantes terá o esquema?")
    k=input("Quantos deles serão necessários para recuperar o segredo?")
    return (n, k, g, p, a0) 

# Coeficientes do polinómio
def Coeficientes(p, a0, n, k):
    """
Função que invoca a escolha dos coeficientes.
Input: recebe o primo p, o segredo, o número de participantes
e o número de elementos para recuperar.
Output: retorna os coeficientes do polinómio interpolador

Uso: o parâmetro a, corresponde aos k-1 números aleatórios.
    """
    Zp= IntegerModRing(p)
    t=k-1
    coef = [a0]
    for r in range (1, t+1):
        a = Zp.random_element() 
        coef.append(a) 
    return coef 

# Construção do polinómio
def polinomio(coef):
    """
Função que invoca a geração do polinómio interpolador de lagrange.
Input: recebe os coeficientes.
Output: retorna o polinómio.

Exemplo: suponhamos que temos o segredo 2000, k= 2. Escolhemos k-1=1
coeficiente [2000, 16749784847526677112774608782480510392], então o polinómio
é: P(x)= 16749784847526677112774608782480510392*x + 2000
    """
    Zp=coef[0].parent()
    Pol.<x> = PolynomialRing(Zp)
    polinomio=Pol(coef)
    return polinomio

# Partes à partilhar
def shared(n, polinomio):
    """
Função que invoca o cálculo das partes secretas.
Input: recebe n e o polinómio interpolador de Lagrange.
Output: retorna pares ordenados (1, polinomio(1)),...,(n, polinomio(n))

Uso: calcula-se as as imagens dos n números na função polinómial,
e retorna-se os pares ordenados secretos, e são distribuidos.

Exemplo: Seja bits= 128, suponhamos que queremos compartilhar 2000, as
partes secretas compartilhadas são:  [[1, 137624036525743829578287975035514517986], 
[2, 275248073051487659156575950071029033972], [3, 412872109577231488734863925106543549958]]
    """
    pares=[]
    for x in range (1, n+1):
        pares.append([x, polinomio(x)])
    return pares

# Recuperação do polinómio
def pol_rec(coef, pares):
    """
Função que invoca a recuperação do segredo compartihado.
Input: recebe k e as partes secretas compartilhadas.
Output: retorna o polinómio interpolador.

Uso: calcula-se os polinómios de Langrange lj e em
seguida, calcula o polinómio interpolador pol.
então, a0 de pol é o segredo reconstruido.
    """
    Zp=coef[0].parent()
    Pol.<x> = PolynomialRing(Zp)
    k = len(coef)
    pol = Pol(0)
    for j in range(k):
        lj = Pol(Zp(1))
        for i in range(k):
            if j!=i:
                lj = lj * ((x-pares[i][0])/(pares[j][0] - pares[i][0]))
        pol = pol + lj * pares[j][1]
    return pol

# Calculo das potencias
i=0
pot=[]
def potencia (g, coef):
    """
Função que invoca o cálculo das potências
(parâmetros públicos) para a verificação.
Input: recebe o gerador e os coeficientes do polinómio.
Output: retorna as potencias.

Uso: calcula-se e distribui-se as k potencias a todos os
participantes.
    """
    pot=[]
    k=len(coef)
    for i in range (k):
        pot.append(g**coef[i])
    return pot

# Verificação da consistência das partes secretas
def feldman_verify (i, g, pot, secret):
    """
Função que invoca a verificação da consistência das partes
secretas de cada interveniente.
Input: recebe i-posição de cada participante, o gerador e as potencias.
Output: retorna dois valores, True ou False, caso positivo ou negativo, respetivamente.

Atenção: Se o resultado for False, implica dizer que, o responsável pela partilha
distribuiu maliciosamente a parte secreta ou por engano assim, a fez.
    """
    prod=1
    k=len(pot)
    for j in range (k):
        prod =prod*(pot[j]**(i**j))
    if g**secret == prod:
        return True
    else:
        return False
    
# Verificar a unicidade do polinomio interpolador de Lagrange
def verify_pols(pol_c, pol_r):
    """
Função que invoca a verificação do teorema da unicidade do polinómio
interpolador de Lagrange.
Input: o polinómio construido e o reconstruído.
Output: True ou False
    """
    if pol_c== pol_r:
        return True
    else:
        return False